In [8]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('reviews_row.csv', encoding='utf-8',
                     skip_blank_lines=True  )
#documents = [' '.join(i[0].split(' ')[1:]) for i in article_data.values]
#print(documents[1
print(df)
#   NaN 없애는거 찾아야됌
df.info()


         순번                            과목명               담당교수   성적부여   평점  \
0     10001                  4차 산업혁명과 융합기술                이응석  Grade  NaN   
1     10002  4차산업/직업과 인체 뇌 기능의 생활 응용 융합 강좌  김영철, 김진하, 김찬형, 최웅  Grade  NaN   
2     10003                       갈등사회와 시민      김남준, 김민재, 조일수  Grade  4.4   
3     10003                       갈등사회와 시민      김남준, 김민재, 조일수  Grade  4.4   
4     10003                       갈등사회와 시민      김남준, 김민재, 조일수  Grade  4.4   
...     ...                            ...                ...    ...  ...   
6339  10211                   환경과 곤충산업의 이해                김재용  Grade  2.5   
6340  10211                   환경과 곤충산업의 이해                김재용  Grade  2.5   
6341  10211                   환경과 곤충산업의 이해                김재용  Grade  2.5   
6342  10211                   환경과 곤충산업의 이해                김재용  Grade  2.5   
6343  10212                  환경과 그린에너지의 이해                김재용  Grade  NaN   

       과제  조모임   학점 비율    출결 시험 횟수  추천수  \
0     NaN  NaN     NaN   NaN   N

In [11]:
index = df['추천수']

In [12]:
print(index)

0       NaN
1       NaN
2       0.0
3       0.0
4       2.0
       ... 
6339    0.0
6340    0.0
6341    0.0
6342    1.0
6343    NaN
Name: 추천수, Length: 6344, dtype: float64


In [2]:
df['과목명'].value_count()


NameError: name 'df' is not defined

In [3]:
import math
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

v1,v2 = [3, 45, 7, 2], [2, 54, 13, 15]
print(v1, v2, cosine_similarity(v1,v2))

[3, 45, 7, 2] [2, 54, 13, 15] 0.97228425171235


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer


text = ['고등학교 때 윤리와 사상이나 생활과 윤리 재밌게 들었다면 수업에 흥미를 느끼실 수 있어요! 과제는 김민재 교수님 출석과제랑 기말글 있었고 시험은 공부한 만큼, 수업 열심히 정리한 만큼 풀 수 있도록 문제 내주셨어요~', # Doc[0] 
        '처음 겪어보는 팀티칭 수업이었는데 나름 신선하고 나쁘지 않은 것 같아요. 배울 점은 솔직히 사람마다 다를 것 같네요. 평소에 독서 많이 하는 사람은 이미 다 아는 내용일수도..?', # Doc[1]
        '사실 내용도 별로 재미 없었고 시험도 무슨 기준으로 채점되는지 모르겠고, 내 점수도 무슨 기준으로 채점된건지 모르겠음. 비율 채워주는 것 같지도 않고(내가 못한건가) 교수님들은 좋은데 수업내용이랑 시험이 별로였어요. 전 시험이 너무 짜증나서 다시는 듣기 싫지만 윤리, 사회 좋아하시면 듣는 것도 나쁘진 않을 것 같아요.',
        '사회학 관심있으면 무조건.... 제발 무조건 들어줘요.... 세 교수님 모두 강의력도 좋으셨어요.',
        '첫 개설 강의라 솔직히 걱정 많이 했는데, 결과적으로 듣길 잘했다는 생각이 든 강의였습니다. 3분 교수님 다 너무 좋으세요. 김민재 교수님이 주교수님이셨는데 정말 꼼꼼하게 시험이나 수업 관련 정보를 알려주십니다. 학생들 배려하시는 모습이 너무 잘 보였어요. 강의력은 다 좋으세요! 전 특히 김남준 교수님 수업이 너무 좋았어요! 90분 조금 넘는 수업인데 예시나 설명이 너무 좋아서 지루하지 않게 잘 들었습니다. 세 분 교수님 다 강의에 열정적인게 보였어요. 과제는 김민재 교수님만 내주셨는데 어렵지 않고, 자기 생각 쓰는거라 부담이 거의 없었어요. 시험은 오픈북으로 다소 신기한 방법으로 봤는데, 설명을 워낙 잘 해주시고 공지를 많이 해주셔서 무난하게 잘 치뤘습니다. 시험 문제 난이도도 괜찮았어요. 기말과제로는 레포트 쓰기 한 개가 있는데, 첫주에 이미 내용이나 형식을 다 공개하셨기 때문에 미리미리 하시면 전혀 부담될거 없으실 겁니다. 얻은 것도 많고, 특히 교양인으로서 한 걸음 나아간 유익한 시간이었어요! 관심 있으신 분들은 꼭 듣는거 추천합니다!'
        ] # Doc[5]

tfidf_vectorizer = TfidfVectorizer() # TF-IDF 객체선언



In [51]:
tfidf_vectorizer.fit(text) # 단어를 학습시킴
tfidf_vectorizer.vocabulary_ # 단어사전을 출력
sorted(tfidf_vectorizer.vocabulary_.items()) # 단어사전 정렬



[('3분', 0),
 ('90분', 1),
 ('강의라', 2),
 ('강의력도', 3),
 ('강의력은', 4),
 ('강의에', 5),
 ('강의였습니다', 6),
 ('같네요', 7),
 ('같아요', 8),
 ('같지도', 9),
 ('개가', 10),
 ('개설', 11),
 ('거의', 12),
 ('걱정', 13),
 ('걸음', 14),
 ('겁니다', 15),
 ('것도', 16),
 ('겪어보는', 17),
 ('결과적으로', 18),
 ('고등학교', 19),
 ('공개하셨기', 20),
 ('공부한', 21),
 ('공지를', 22),
 ('과제는', 23),
 ('관련', 24),
 ('관심', 25),
 ('관심있으면', 26),
 ('괜찮았어요', 27),
 ('교수님', 28),
 ('교수님들은', 29),
 ('교수님만', 30),
 ('교수님이', 31),
 ('교양인으로서', 32),
 ('기말과제로는', 33),
 ('기말글', 34),
 ('기준으로', 35),
 ('김남준', 36),
 ('김민재', 37),
 ('꼼꼼하게', 38),
 ('나름', 39),
 ('나쁘지', 40),
 ('나쁘진', 41),
 ('나아간', 42),
 ('난이도도', 43),
 ('내가', 44),
 ('내용도', 45),
 ('내용이나', 46),
 ('내용일수도', 47),
 ('내주셨는데', 48),
 ('내주셨어요', 49),
 ('너무', 50),
 ('넘는', 51),
 ('느끼실', 52),
 ('다를', 53),
 ('다소', 54),
 ('다시는', 55),
 ('독서', 56),
 ('듣기', 57),
 ('듣길', 58),
 ('듣는', 59),
 ('듣는거', 60),
 ('들어줘요', 61),
 ('들었다면', 62),
 ('들었습니다', 63),
 ('때문에', 64),
 ('레포트', 65),
 ('만큼', 66),
 ('많고', 67),
 ('많이', 68),
 ('모두', 69),
 ('모르겠고', 70),

In [52]:
tfidf_vectorizer.idf_.shape

(184,)

In [57]:
tfidf_vectorizer.transform(text).toarray()[0]
# 여기서 L2정규화 까지 해야 TF-IDF가 완성

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.19,
       0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.13, 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.19, 0.  , 0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.19, 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.  ,
       0.39, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.  , 0.  , 0.  , 0.19,
       0.  , 0.  , 0.  , 0.16, 0.  , 0.19, 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.16, 0.19, 0.  , 0.  , 0.19, 0.19, 0.19,
       0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.

In [61]:
from sklearn import preprocessing 
#count_vec -> tf벡터
tfidf_vectorizer.idf_ = tfidf_vectorizer.idf_.reshape(1, -1)
tfidf_vectorizer = preprocessing.normalize(tfidf_vectorizer, norm='l2')


TypeError: object of type 'NoneType' has no len()

In [12]:
# t is token
# d is document == tokens
def f(t, d):
    return d.count(t)

def tf(t, d):
    return 0.5 + 0.5*f(t,d)/max([f(w,d) for w in d])

def idf(t, D):
    # D is documents == document list
    numerator = len(D)
    denominator = 1 + len([ True for d in D if t in d])
    return log10(numerator/denominator)

In [15]:
#1. TF벡터와 IDF벡터를 원소곱 해줍니다.
#2. 위의 결과에 L2 정규화를 해줍니다.
#L2 정규화란??-> 각각의 원소들을 제곱합에 루트를 씌워준 값으로 나눠준 것입니다.
# 벡터화의 결과를 좀 더 완만하게 만들어줌 

In [55]:
tfidf_vectorizer.transform(text).toarray()[0]
#첫번째  TF -IDF 벡터
#첫 번재 문서(문장)에서 나오는 단어 빈도수 벡터

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.19,
       0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.13, 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.19, 0.  , 0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.19, 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.  ,
       0.39, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.  , 0.  , 0.  , 0.19,
       0.  , 0.  , 0.  , 0.16, 0.  , 0.19, 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.16, 0.19, 0.  , 0.  , 0.19, 0.19, 0.19,
       0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.

In [19]:
tfidf_vectorizer.transform(text).toarray()
#이게 결과물임

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.19479448,
        0.        , 0.19479448, 0.        , 0.15715896, 0.        ,
        0.        , 0.        , 0.        , 0.13045613, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.19479448,
        0.        , 0.        , 0.15715896, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.19479448,
        0.        , 0.        , 0.19479448, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.19479448, 0.        , 0.        ,
        0.        , 0.38958896, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [56]:
tfidf_vectorizer.transform(text).toarray()



array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.19,
        0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.13, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.19, 0.  , 0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.19, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.  ,
        0.39, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.  , 0.  , 0.  , 0.  , 0.19,
        0.  , 0.  , 0.  , 0.16, 0.  , 0.19, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.16, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.16, 0.19, 0.  , 0.  , 0.19, 0.19, 0.19,
        0.  , 0.  , 0.  , 0.  , 0.19, 

In [6]:
import numpy as np
np.set_printoptions(precision=2)
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfindVectorizer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())


rank = tfidf_vectorizer.transform(text).toarray()
tfidf = TfidfVectorizer()
sent_graph = np.dot(tfidf_mat, rank.T)
sent_graph.shape


ImportError: cannot import name 'TfindVectorizer' from 'sklearn.feature_extraction.text' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py)

In [5]:
from math import log10
# 구현된거....
# =======================================
# -- TF-IDF function
# =======================================
def f(t, d):
    # d is document == tokens
    return d.count(t)

def tf(t, d):
    # d is document == tokens
    return 0.5 + 0.5*f(t,d)/max([f(w,d) for w in d])

def idf(t, D):
    # D is documents == document list
    numerator = len(D)
    denominator = 1 + len([ True for d in D if t in d])
    return log10(numerator/denominator)

def tfidf(t, d, D):
    return tf(t,d)*idf(t, D)

def tokenizer(d):
    # return [ t for t in d.split() if len(t) > 1 ]
    return d.split()

def tfidfScorer(D):
    tokenized_D = [tokenizer(d) for d in D]
    result = []
    for d in tokenized_D:
        result.append([(t, tfidf(t, d, tokenized_D)) for t in d])
    return result

for i, doc in enumerate(tfidfScorer(corpus)):
    print('====== document[%d] ======' % i)
    print(doc)

NameError: name 'corpus' is not defined

In [35]:
import pandas as pd
rating_data = pd.read_csv('./ratings_good_and_score.csv')
lecture_data = pd.read_csv('./lecture_jumsu.csv')
rating_data.head()
#lecture_data.head()

,id,lecture_index,rating,score
0,7,10020,1.0,3.6
1,1,10089,0.9,4.5
2,1,10060,0.8,4.3
3,2,10049,0.9,4.2
4,2,10018,0.7,4.1


In [36]:
print(rating_data.shape)
print(lecture_data.shape)

(13, 4)
(212, 30)


In [62]:
#pd.merge를 할려면 id가 중복되거나 lecture가 중복되어야 함 -> 여기서 사용자-강의 평점 기반이냐, 강의 정보 기반이냐로 갈림
#우리는 강의기반으로 merge로 통합
user_lecture_data = pd.merge(lecture_data,rating_data,  on = 'lecture')
user_lecture_data.head()

KeyError: 'lecture'